In [4]:
import json
import requests
from pprint import pprint
from bs4 import BeautifulSoup as bs

In [1]:
from webview.models import Professor, Subject

In [40]:
#Adding Faculty to database 
def getFacultyList()  :
    """
    Retruns list of dicts : 
    Same DICT : 
        {'department': 'Aerospace Engineering',
         'dept_code': 'AE',
         'designation': 'Assistant Professor',
         'encrypt_key': 'ae-ap',
         'faculty': '<a href="department/AE/faculty/ae-ap">Akshay Prakash</a>'}
    """
    
    url = "http://www.iitkgp.ac.in/facultylist"

    querystring = {"processOn":"onload","colName":"","searchContent":"","_":"1546070278570"}

    headers = {
        'Cookie': "_ga=GA1.3.1618811768.1543517854; JSESSIONID=0830812A57CAE824DAB209654E57B4EE",
        'Accept-Encoding': "gzip, deflate",
        'Accept-Language': "en-IN,en-US;q=0.9,en;q=0.8",
        'User-Agent': "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_14_1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/71.0.3578.98 Safari/537.36",
        'Accept': "application/json, text/javascript, */*; q=0.01",
        'Referer': "http://www.iitkgp.ac.in/faclistbydepartment",
        'X-Requested-With': "XMLHttpRequest",
        'Connection': "keep-alive",
        'cache-control': "no-cache",
        'Postman-Token': "6be29381-c68c-40f6-a10d-e63045691718"
        }

    response = requests.request("GET", url, headers=headers, params=querystring)

    return json.loads(response.text)["data"]

def getFacultyUrl(data) : 
    """
    Return faculty name, url 
    Sample Argument : '<a href="department/AE/faculty/ae-ap">Akshay Prakash</a>'
    """
    BASE_URL = "http://www.iitkgp.ac.in/"
    soup = bs(data)
    tag = soup.find("a")
    name = tag.text
    url = BASE_URL + tag["href"]
    return name, url


def addFaculty(name, depCode, url, designation) : 
    data = Professor() 
    data.name = name
    data.departmentCode = depCode
    data.website = url
    data.designation = designation 
    data.save()

In [38]:
facList = getFacultyList()
FacultyDepartmentDict = dict() #This dictionary keeps the mapping of dep code to department name of 

In [41]:
for faculty in facList : 
    name, url = getFacultyUrl(faculty["faculty"])
    FacultyDepartmentDict[faculty["dept_code"]] = faculty["department"]
    addFaculty(name, faculty["dept_code"], url, faculty["designation"])

In [2]:
def addSubject(subCode, subName, credits) : 
    data = Subject()
    data.code = subCode
    data.name = subName 
    data.credits = credits
    data.save()

In [5]:
#Adding subjects
with open("/Volumes/Data/CODE/test/rateMyProf/result.json", "r") as f : 
    """
    Format : 
        'AE21001': ['INTRODUCTION TO AERODYNAMICS', '4'],
        Sub_CODE : [SUB NAME, Credits]
    """
    subjectList = json.loads(f.read())

In [6]:
for subCode, li in subjectList.items() : 
    addSubject(subCode, li[0], int(li[1]))